# **Lateral Point Spread function Quality Control Test**
---
The lateral point spread function test is used to determine the full width half maximum X and Y resolution of the micrsoscope. This notebook uses a localiser to detect multiple fluoresent beads in a given field of view to determine an average value for the FWHM in the X and Y direction along side teh standard deviation. Thus, given a statistical output to the systems resolution multiple measurements.

In [3]:
#!Pip install tifffile
#!pip install fpdf
#!pip install matplotlib
#!pip install pandas
#!pip install scikit_image
#!pip install ipywidgets
#!pip install jupyterlab anywidget
#!pip install plotly anywidget
#!pip install -U kaleido

In [4]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [2]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
import pandas as pd
from fpdf import FPDF
from skimage.feature import peak_local_max
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter

class Localiser:
    def __init__(self, image, threshold=200, sigma=0.5, spot_size=10):
        self.raw_image = image
        self._threshold = threshold
        self._sigma = sigma
        self._spot_size = spot_size
        
    def _background_correction(self):
        background = np.median(self.raw_image, axis=0)
        return self.raw_image - background
    
    def _denoise_image(self):
        # denoise background corrected image
        return gaussian_filter(self._background_correction(), self._sigma)
    
    def _gaussian_2D(self, xy, A, x0, y0, sigma_x, sigma_y):
        # 2D Gaussian function
        x,y = xy
        x0 = float(x0)
        y0 = float(y0)
        g = A*np.exp( - (((x-x0)**2)/(2*sigma_x**2) + ((y-y0)**2)/(2*sigma_y**2)))
        return g.ravel()

    def spot_detector(self):
        denoised_image = self._denoise_image()
        # Perform spot detection using peak_local_max
        spot = peak_local_max(denoised_image, min_distance=30, threshold_abs=self._threshold, exclude_border=True)
        return spot
    
    def get_localiser_spot_parameters(self):
        spots=self.spot_detector()
        fitted_spots = []
        fitted_intensities = []
        centroids = []
        std_deviations = []
        
        for spot in spots:
            # get spot positions and create meshgrids 
            x, y = spot[1], spot[0]
            x_range = np.arange(x - self._spot_size, x + self._spot_size, 1)
            y_range = np.arange(y - self._spot_size, y + self._spot_size, 1)
            spot_box_intensities = self.raw_image[y - self._spot_size : y + self._spot_size, x - self._spot_size : x + self._spot_size]
            x_mesh, y_mesh = np.meshgrid(x_range, y_range)
           
            # guess initial Gaussian Paramters
            p0 = [np.max(spot_box_intensities), x, y, self._spot_size/2, self._spot_size/2]
            param,_ = curve_fit(self._gaussian_2D, (x_mesh, y_mesh), spot_box_intensities.ravel(), p0)
            fitted_spots.append(param)
        
            # Evaluate the fitted Gaussian function
            fitted_intensity = self._gaussian_2D((x_mesh, y_mesh), *param)
            fitted_intensity = fitted_intensity.reshape(spot_box_intensities.shape)
            fitted_intensities.append(fitted_intensity)
            
            # get standard dev
            sigma_x, sigma_y = param[3], param[4]
            std_deviations.append([sigma_x, sigma_y])
                        
            centroid_x = x
            centroid_y = y
            centroids.append([centroid_y, centroid_x])
            
            
            
        return [fitted_spots, fitted_intensities, centroids , std_deviations]

## Create a template of the PDF required for tests

def build_pdf(report_path, figure_path, QC_results, QC_microscope_parameters):
    class PDF(FPDF):
        def header(self):
            # Set font and size for the header
            self.set_font("helvetica", "B", 16)
            self.cell(0, 20, title , align='C', ln=1)

        def footer(self):
            # Page number in the footer
            self.set_y(-15)
            self.set_font("helvetica", "I", 8)
            self.cell(0, 10, f"Page {self.page_no()}", align='C', ln=1)

    title = "Quality Control Report"
    pdf = PDF()
    print("Created the PDF file")
    pdf.add_page()
    # Set font and size for the table 
    pdf.set_title(title)
    
    # Set font and size
    pdf.set_font("helvetica", "B", 12)
    pdf.cell(0, 6, "QC Test: " + QC_microscope_parameters['QC test'], 0 , 1, "L")
    pdf.cell(0, 6, "QC Date: " + QC_microscope_parameters['QC date'],  0 , 1, "L")
    pdf.ln(2)
    
    pdf.set_font("helvetica", "B", 14)
    pdf.cell(0, 12, "Microscope Parameters:", 0 , 2, "L")
    
    pdf.set_font("helvetica", "B", 10)

    pdf.cell(0, 6, "Microscope: " + QC_microscope_parameters['Microscope'],  0 , 1, "L")
    pdf.cell(0, 6, "Imaging Modality: " + QC_microscope_parameters['Imaging mode'],  0 , 1, "L")
    pdf.cell(0, 6, "Camera: " + QC_microscope_parameters['Camera'], 0 , 1, "L")
    pdf.cell(0, 6, "Objective lens: " + QC_microscope_parameters['Objective lens'], 0 , 1, "L")
    pdf.ln(3)

                                                                                                                                                                                                
    # Add results section heading
    pdf.set_font("helvetica", "B", 14)
    pdf.cell(0, 12, QC_microscope_parameters['QC test'] + " Results:",  align='L', ln=1)
    pdf.ln(1)

    # Draw table header with bold text
    pdf.set_font("helvetica", "B", 10)
    for header in QC_results.columns:
        pdf.cell(45, 6, header, border=2)
    pdf.ln()
    
    # Set font and size for the table content
    pdf.set_font("helvetica", "", 10)

    # Add the table data
    for _, row in QC_results.iterrows():
        for item in row:
            pdf.cell(45, 6, str(item), border=1)
        pdf.ln()
        
    # Add a Matplotlib figure with imshow and colorbar
    if not os.path.exists(figure_path):
        print("image not exist")
    pdf.image(figure_path, x=20, y=120, w=160)

    # Output the PDF
    pdf.output(report_path)

    
# Utiliy function for used throughout the notebook
def generate_file_path(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)
        
def check_and_create_new_file_name(file_path):
    base, ext = os.path.splitext(file_path)
    # Initialize a suffix
    suffix = 0

    # Create a new file path with a suffix and check if it exists
    new_file_path = f"{base}{ext}"
    
    while os.path.exists(new_file_path):
        suffix += 1
        new_file_path = f"{base}-{suffix}{ext}" 
        print("The file exist.")
        print("Created a new file path: "+new_file_path)
    return new_file_path

        

In [3]:
        
## create widgets to create metadata     
import ipywidgets as widgets
from IPython.display import display , clear_output
from PIL import Image as PILImage
from io import BytesIO

# Create a file dialog widget that only accepts TIFF files
file_dialog = widgets.FileUpload(
    accept='.tiff, .tif',  # Allow only TIFF files
    multiple=False  # Allow only single file selection
)

# Define a variable to store the file path
uploaded_file_path = None
QC_images = None

filedialog_output = widgets.Output()

# Define a function to handle the uploaded TIFF files
def handle_upload(change):
    global QC_images
    
    uploaded_files = file_dialog.value
    print(uploaded_files)
    if uploaded_files:
        # Get the first uploaded file's information
        first_uploaded_file_key = next(iter(uploaded_files))
        file_info = uploaded_files[first_uploaded_file_key]
        
        # Access metadata and content from the file_info dictionary
        file_name = file_info['metadata']['name']
        file_content = file_info['content']
        
        # Save the file content to a BytesIO object for later processing (optional)
        file_content_io = BytesIO(file_content)
        
        # Process the uploaded file as needed
        QC_images = tifffile.imread(file_content_io)
        print(f"Uploaded TIFF file: {file_name}")
    else:
        print("No files uploaded.")
     
    
# Define the dropdown widget
dropdown_microscope = widgets.Dropdown(
    options=['AdvancedSpinningDisk', 'BasicSpinningDisk', 'LeicaSp8'],
    description='Select a Microscope:',
    style={'description_width': 'initial'},
    value=None
)

dropdown_imaging_modality = widgets.Dropdown(
    options=['Confocal', 'Widefield'],
    description='Imaging Mode:',
    style={'description_width': 'initial'},
    value=None
)

dropdown_camera = widgets.Dropdown(
    options=["Flash4", "Prime", "Kinetix","PMT","Hyd"],
    description='Camera:',
    style={'description_width': 'initial'},
    value=None
)    

dropdown_objective_lens = widgets.Dropdown(
    options=['10xAir', '20xAir','40xAir', '40xOil', '63xOil', '63xGly'],
    description='Objective_lens:',
    style={'description_width': 'initial'},
    value=None
)    

# Create a text box widget for entering the 
QC_date_selector = widgets.DatePicker(
    description='Date of QC',
    disabled=False
)       

# Define the output widgets for selected options
selected_microscope_output = widgets.Output()
selected_imaging_modality_output = widgets.Output()
selected_objective_lens_output = widgets.Output()
selected_camera_output = widgets.Output()

# Define the variable to store the selected option
selected_microscope_string = None
selected_imaging_modality_string = None
selected_objective_lens_string = None
selected_camera_string = None



# Create a text box widget for entering the 
QC_date_selector = widgets.DatePicker(
    description='Date of QC',
    disabled=False
)

# Create a text box widget for entering the directory path
directory_textbox_savepath = widgets.Text(
    value='',  # Initial value
    placeholder='Enter Folder path',  # Placeholder text
    description='Save Directory',  # Label displayed next to the text box
    style={'description_width': 'initial'}

)

# Create a text box widget for entering the directory path
directory_textbox_custom_QC_filename = widgets.Text(
    value='',  # Initial value
    placeholder='(Optional)',  # Placeholder text
    description='Enter QC report name (Optional)',  # Label displayed next to the text box
    style={'description_width': 'initial'}

)

# set ipywidgets for data capture
# Create a text box widget for entering the directory path
directory_guassian_sigma = widgets.Text(
    value='2',  # Initial value
    placeholder='enter value',  # Placeholder text
    description='Guassian Blur Sigma',  # Label displayed next to the text box
    style={'description_width': 'initial'},

)

## Modified by Queenie
# Create a file dialog widget that only accepts TIFF files fro SP8 only
file_dialog_SP8 = widgets.FileUpload(
    accept='.tiff, .tif',  # Allow only TIFF files
    multiple=False  # Allow only single file selection
)

# Define a variable to store the file path
uploaded_file_path = None
QC_images = None

filedialog_SP8_output = widgets.Output()
# Define a function to handle the uploaded TIFF files for SP8
def handle_upload_SP8(change):
    global QC_images
    with filedialog_SP8_output:
        uploaded_files = file_dialog_SP8.value ##different for SP8 tiff
        
        # Get the first uploaded file's information
        first_uploaded_file_key =next(iter( uploaded_files))
        ##print(first_uploaded_file_key)
                
        file_info = first_uploaded_file_key

        # Access metadata and content from the file_info dictionary
        file_name = file_info['name']
        file_content = file_info['content']
        
        # Save the file content to a BytesIO object for later processing (optional)
        file_content_io = BytesIO(file_content)
        
        # Process the uploaded file as needed
        QC_images = tifffile.imread(file_content_io)
        print(f"Uploaded TIFF file: {file_name}")


##define button widget to update
button_send = widgets.Button(
                description='UPdATE SETTINGS and IMAGES',
                tooltip='Send',
                style={'description_width': 'initial'}
            )
output = widgets.Output()
QC_microscope_parameters = None
def on_button_clicked(event):
    with output:
        clear_output()
        global QC_microscope_parameters
        print("Updates settings: ")
        # add test paramters
        global QC_date 
        QC_date = str(QC_date_selector.value)
        selected_microscope = dropdown_microscope.value
        selected_imaging_modality = dropdown_imaging_modality.value
        selected_camera = dropdown_camera.value
        selected_objective_lens= dropdown_objective_lens.value
        
        QC_microscope_parameters = {"QC test": "xy_PSF",
                 "QC date": QC_date,
                 "Microscope": selected_microscope,
                 "Imaging mode": selected_imaging_modality,
                 "Camera": selected_camera,
                 "Objective lens": selected_objective_lens
        }
        
        print(QC_microscope_parameters)
      
        ## Modified by Queenie  
        if "LeicaSp8" in selected_microscope:
            ##print("LeicaSP8")
  
            # Display the file dialog widget
            display(file_dialog_SP8, filedialog_SP8_output)

        else:
            print("Slidebook")


            # Display the file dialog widget
            display(file_dialog)

#ICR path drive will be where your RDS is mapped - Z:\DSR\LMFACCHE\QC\Maintenance(delete if sharing)

button_send.on_click(on_button_clicked)
vbox_result = widgets.VBox([button_send, output])
# Attach the function to the file dialog's value change
file_dialog_SP8.observe(handle_upload_SP8, names='value')
# Attach the function to the file dialog's value change
file_dialog.observe(handle_upload, names='value')

## ** Set Microscope Parameters**
---

In [4]:
# Display the dropdowns and the selected option outputs
display(dropdown_microscope)
display(selected_microscope_output)

display(dropdown_imaging_modality)
display(selected_imaging_modality_output)

display(dropdown_camera)
display(selected_camera_output)  

display(dropdown_objective_lens)
display(selected_objective_lens_output)  

# Display the text box widget
display(QC_date_selector)

# Display the text box widget
display(directory_textbox_savepath)
# Display the text box widget
display(directory_textbox_custom_QC_filename)


display(vbox_result)



Dropdown(description='Select a Microscope:', options=('AdvancedSpinningDisk', 'BasicSpinningDisk', 'LeicaSp8')…

Output()

Dropdown(description='Imaging Mode:', options=('Confocal', 'Widefield'), style=DescriptionStyle(description_wi…

Output()

Dropdown(description='Camera:', options=('Flash4', 'Prime', 'Kinetix', 'PMT', 'Hyd'), style=DescriptionStyle(d…

...

Dropdown(description='Objective_lens:', options=('10xAir', '20xAir', '40xAir', '40xOil', '63xOil', '63xGly'), …

Output()

DatePicker(value=None, description='Date of QC', step=1)

Text(value='', description='Save Directory', placeholder='Enter Folder path', style=TextStyle(description_widt…

Text(value='', description='Enter QC report name (Optional)', placeholder='(Optional)', style=TextStyle(descri…

##  **Set localiser Settings and Run Lateral Point Spread Function Control Test.**
It will generate the QC report.
The QC_reports will be under the folder and will have the micrscope data generated in the report name e.g **2023-09-08_LeicaSp8_40xAir_Confocal_Flash4_QC_report.pdf**. This will not be deleted if file already exists with the name a new file with be generated **xxx_QC_report-1.pdf** etc.00
    
Lastly, the CSV will be generated if it does not already exist, and if one does currenlty exist, it will append the microscope details, QC results and location of the corresponding QC PDF report.


In [19]:
#

from ipywidgets import Output
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Create a text box widget for entering the directory path
directory_textbox1 = widgets.Text(
    value='400',  # Initial value
    placeholder='enter value',  # Placeholder text
    description='Localiser Threshold',  # Label displayed next to the text box
    style={'description_width': 'initial'},

)

directory_textbox2 = widgets.Text(
    value='2',  # Initial value
    placeholder='enter value',  # Placeholder text
    description='Guassian Blur Sigma',  # Label displayed next to the text box
    style={'description_width': 'initial'},

)


directory_textbox3 = widgets.Text(
    value='10',  # Initial value
    placeholder='enter value',  # Placeholder text
    description='Spot Size',  # Label displayed next to the text box
    style={'description_width': 'initial'},

)
    
directory_textbox4 = widgets.Text(
    value='9.86',  # Initial value
    placeholder='enter value',  # Placeholder text
    description='pixel to microns',  # Label displayed next to the text box
    style={'description_width': 'initial'},

)

#define function for show images 
def show_images(images):
    plt.imshow(images, cmap='gray')  # Use 'gray' colormap for grayscale images
    plt.title("QC Image")
    plt.colorbar()  # Show a colorbar if needed
    plt.show()

       
out2 = Output()
 
def generate_filepath(QC_microscope_parameters,directory_textbox_savepath):
    with out2:
        ## generate the file path
        base_directory = directory_textbox_savepath.value
        file_path = base_directory + "\\" +  QC_microscope_parameters["Microscope"] + '\\' + QC_microscope_parameters["QC test"] + "\\"
        generate_file_path(file_path)

        # Generate csv name and directory path
        csv_save_name = QC_microscope_parameters["Microscope"] + "_" + QC_microscope_parameters["QC test"] + ".csv"
        csv_save_path = file_path + csv_save_name

        # Generate temp file of QC test results to load into PDF
        figure_path_temp = base_directory + "\\" +"temp_files\\"
        generate_file_path(figure_path_temp)
        figure_path_name = figure_path_temp + 'temp.png'

        # Generate csv name and directory path
        report_path = file_path + "Quality_Control_Reports"
        generate_file_path(report_path) ##generate folder
        report_name = "\\" + QC_date + "_" + QC_microscope_parameters["Microscope"] + "_" + QC_microscope_parameters["Objective lens"] + "_" + QC_microscope_parameters["Imaging mode"] + "_" + QC_microscope_parameters["Camera"]  + '_QC_report.pdf'
        ##if exist(odirectory_textbox_custom_QC_filename.value):
            ##report_full_path = check_and_create_new_file_name(report_path + directory_textbox_custom_QC_filename.value) 

        try:
            # Try to use the variable
            print(odirectory_textbox_custom_QC_filename.value)  # Or any other operation with my_variable
        except NameError:
            # Handle the case where the variable is not defined
            report_full_path = check_and_create_new_file_name(report_path + report_name)
        else:
            # This block executes if no NameError occurs (variable is defined)
            report_full_path = check_and_create_new_file_name(report_path + directory_textbox_custom_QC_filename.value)
      
        print(report_full_path)
        return [report_full_path, figure_path_name, report_name,csv_save_path]

  

## define the localisation function
def calculate_FWHM(sigmas: int, pixel_to_um):
    fwhm_y = []
    fwhm_x = []
    for sigma in sigmas:
        fwhm_y.append(sigma[0]*2.355/pixel_to_um)
        fwhm_x.append(sigma[1]*2.355/pixel_to_um)
    
    return [fwhm_y, fwhm_x]

## fucniton using plotly
def create_or_update_scatter(fwhm_x, fwhm_y, fig=None):
    """Creates or updates a scatter plot using Plotly."""

    magnitudes = np.sqrt(np.array(fwhm_x)**2 + np.array(fwhm_y)**2)

    if fig is None:  # Create a new figure
        
        fig = go.FigureWidget(data=[go.Scatter(
            x=fwhm_x,
            y=fwhm_y,
            mode='markers',
            marker=dict(
                size=4,
                color=magnitudes,  # Color based on magnitudes
                colorscale='Viridis'  # Use the Viridis color scale
            )
        )])

        fig.update_layout(
            title="FWHM Y vs FWHM X",
            xaxis_title="microns",
            yaxis_title="microns",
            width = 700,
            height = 500
            #margin=dict(l=50, r=50, b=50, t=80)  # Adjust margins if needed
        )
        
        return fig # Return the new figure

    else:  # Update an existing figure
        fig.data[0].x = fwhm_x
        fig.data[0].y = fwhm_y
        fig.data[0].marker.color = magnitudes

        # Important: You might need to update layout ranges if data changes drastically
        fig.update_xaxes(range=[min(fwhm_x), max(fwhm_x)]) #Example
        fig.update_yaxes(range=[min(fwhm_y), max(fwhm_y)]) #Example

        return fig # Return the updated figure
        
## fucniton using matplotlib
def create_or_update_matplotlib(fwhm_x, fwhm_y, scatter=None):
    """Creates or updates a scatter plot using Matplotlib."""

    # Create figure and axes *before* displaying anything
    fig, ax = plt.subplots()

    if scatter is None:
        
        scatter = ax.scatter(fwhm_x, fwhm_y,s=4, c=np.sqrt(np.array(fwhm_x)**2+np.array(fwhm_y)**2), cmap='viridis')
        ax.set_title("FWHM Y vs FWHM X")
        ax.set_xlabel("microns")
        ax.set_ylabel("microns")
        plt.tight_layout()  # Important: call tight_layout *before* saving

        fig.canvas.draw()  # Draw the scatter plot
        plt.savefig(figure_path_name, bbox_inches='tight', dpi=200)


    else:
        new_x ,new_y = fwhm_x, fwhm_y
        magnitude = np.sqrt(new_x**2 + new_y**2)  # Calculate colors
        scatter.set_offsets(np.c_[new_x, new_y]) # Update data
        scatter.set_facecolors(plt.cm.viridis(magnitude / np.max(magnitude))) # Normalize colors


        fig.canvas.draw()
        plt.tight_layout()  # Important: call tight_layout *before* saving
        plt.savefig(figure_path_name, bbox_inches='tight', dpi=200)

    return fig



def run_analysis_getdata (images, test_parameters):
    
    if images is not None:
        QC_images = images
        threshold = test_parameters["threshold"]
        sigma = test_parameters["sigma"]
        spot_size = test_parameters["spot_size"]
        pixel_to_microns = test_parameters["pixel_to_microns"]     
        
        # run analysis and generate data
        localiser = Localiser(QC_images, threshold, sigma, spot_size)  ## call the class
        spots = localiser.spot_detector()
        param = localiser.get_localiser_spot_parameters()
        
        fwhm_y, fwhm_x = calculate_FWHM(param[3], pixel_to_microns)
               
        mean_fwhm_y = round(np.mean(fwhm_y), 2)
        mean_fwhm_x = round(np.mean(fwhm_x), 2)
    
        std_dev_fwhm_y = round(np.std(fwhm_y), 2)
        std_dev_fwhm_x = round(np.std(fwhm_x), 2)     
    
        
        # Extracts the data and create dataframes with the QC test. 
        data_report = {
              'Mean FWHM X (um)': [mean_fwhm_x],
              'Mean FWHM Y (um)': [mean_fwhm_y],
              'std dev FWHM X (um)': [std_dev_fwhm_x],
              'Std dev FWHM Y (um)': [std_dev_fwhm_y]
        }
    
        df_QC_report = pd.DataFrame(data_report)
        #print(df_QC_report)
    
        data_csv = {
              'Date of QC (yyyy-mm-dd)': [QC_date],
              'Imaging Modality': QC_microscope_parameters["Imaging mode"],
              'Camera': QC_microscope_parameters["Camera"],
              'Objective lens': QC_microscope_parameters["Objective lens"],
              'Mean FWHM X (um)': [mean_fwhm_x],
              'Mean FWHM Y (um)': [mean_fwhm_y],
              'std dev FWHM X (um)': [std_dev_fwhm_x],
              'Std dev FWHM Y (um)': [std_dev_fwhm_y],
              'QC Report Name': f"{report_name}",
              'Location of PDF Report': f"{report_full_path}"
        }
    
        df_csv = pd.DataFrame(data_csv)
    
    else:
        raise ValueError("Image data is empty")

    return [df_csv, df_QC_report ,fwhm_y, fwhm_x]

##
def generate_pdf_report(df):
    df_csv = df[0]
    df_QC_report = df[1]
    if not os.path.isfile(csv_save_path):
       df_csv.to_csv(csv_save_path, index=False, header='column_names')
    else: # else it exists so append without writing the header
       df_csv.to_csv(csv_save_path, mode='a', header=False, index=False)
    
    ##report_full_path = check_and_create_new_file_name(report_full_path)
    build_pdf(report_full_path, figure_path_name, df_QC_report, QC_microscope_parameters)
    return report_full_path
##


out1 = Output()
def update_localiser_settings():
    with out1:
        global fig, ax, scatter ,fig1
        # add test paramters
        global test_parameters 
        scatter = None
        fig1 = None
        
        print("Updated localiser settings. ")
        
        threshold = int(directory_textbox1.value)
        sigma = float(directory_textbox2.value)
        spot_size = int(directory_textbox3.value)
        pixel_to_microns = float(directory_textbox4.value)
        
       
        test_parameters = {"threshold": threshold,"sigma": sigma,"spot_size" : spot_size,"pixel_to_microns" :pixel_to_microns}      
        print("Running localiser... ")

        #display(button_internal,fig)
        df_csv, df_QCreport, fwhm_y, fwhm_x = run_analysis_getdata (QC_images, test_parameters)
        
        fig1 = create_or_update_scatter(fwhm_x, fwhm_y)
        
        fig = create_or_update_matplotlib(fwhm_x, fwhm_y)
        with out1:  # Display the figure *within* the output widget
            clear_output(wait=True)  # Clear any previous output in the widget
            #display(fig)
            display(fig1)
            display(df_QCreport)
    
            print(figure_path_name)
        generate_pdf_report([df_csv, df_QCreport])
        return fig    
    
        

out = Output()  # Create an Output widget
def show_localisation_function(b):
    global fig, report_full_path, figure_path_name,report_name,csv_save_path
    with out:  # Use the Output widget as a context manager
        clear_output(wait=True)
        
        print("This is my Localiser function running in the notebook!")
        
        show_images(QC_images)
        [report_full_path, figure_path_name,report_name,csv_save_path] = generate_filepath(QC_microscope_parameters,directory_textbox_savepath)
        
        # Display the text box widget
        display(directory_textbox1)
        display(directory_textbox2)
        display(directory_textbox3)
        display(directory_textbox4)
       
        display(button_run_localise)

      

        #return fig





In [20]:
### Define two buttons, Startand Run localiser
test_parameters = None
fig = None


button_internal = widgets.Button(description="Start Localiser")
button_internal.on_click(show_localisation_function)

##define button widget to update
button_run_localise = widgets.Button(
                description='Run localiser (Update SETTINGS.) ',
                tooltip='Send',
                style={'description_width': 'initial'}
            )
button_run_localise.layout.width = 'auto'  # Adjusts to content width
button_run_localise.on_click(lambda b: update_localiser_settings())

hbox_result = widgets.HBox([button_internal, out,out1])  
display(hbox_result)
#display(button_internal)
#display(out)  # Display the Output widget

